In [ ]:
""" Point of this notebook is to demonstrate a method of simiulating a very simple, made up XPS spectrum using cross section"""

**Imports**

In [ ]:
import copy
import itertools as it

import sim_xps_spectra.broad_functs.create_broaden_functs as createBFuncts
import sim_xps_spectra.mol_spectra.standard_objs as molSpecObjs
import sim_xps_spectra.gen_spectra.standard_objs as genSpectraObjs
import sim_xps_spectra.gen_spectra.get_spectrum as genSpectraRunner

import sim_xps_spectra.x_sections.standard_objs as xSectObjs
import sim_xps_spectra.x_sections.yeh_lindau_db as yhLindau

import sim_xps_spectra.plotters.standard_plotters as dPlotters

**Configuration variables**

In [ ]:
#What x-values we want to generate the spectrum at
X_VALS = [x/100 for x in range(100*20)]

#We use the same broadening function for ALL atomic orbitals
FWHM = 0.5
PHOTON_ENERGY = 500

#All variable for one fragment needs to be stored in this object
class MolFragInfo():
    def __init__(self, label ,energies, baseIntensities):
        self.label = label
        self.energies = energies
        self.baseIntensities = baseIntensities


#Define our fragment A
fragAEnergiesCl3s = [5,8]
fragAEnergiesCl3p = [1,3,8]
fragAIntensitiesCl3s = [2,5]
fragAIntensitiesCl3p = [2,7,1]

fragALabelCl3s = molSpecObjs.MolFragLabel(fragKey="Anion", eleKey="Cl", aoKey="3s")
fragALabelCl3p = molSpecObjs.MolFragLabel(fragKey="Anion", eleKey="Cl", aoKey="3p")
fragA_Cl3s = MolFragInfo(fragALabelCl3s, fragAEnergiesCl3s, fragAIntensitiesCl3s)
fragA_Cl3p = MolFragInfo(fragALabelCl3p, fragAEnergiesCl3p, fragAIntensitiesCl3p)

#Define our fragment B

#Put all fragments in a list
ALL_FRAGS = [fragA_Cl3s, fragA_Cl3p]

**Create our broaden functions**

In [ ]:
def createBroadenFunctFromEnergiesIntensitiesAndFWHM(energies:list, intensities:list, fwhm):
    allFuncts = list()
    for e,i in it.zip_longest(energies,intensities):
        currGau = createBFuncts.createNormalisedGauFunctFromCentreAndFWHM(e, fwhm, area=i)
        allFuncts.append(currGau)
    return createBFuncts.BroadenFunctCompositeStandard(allFuncts)
    
allBroadenFuncts = list()
for obj in ALL_FRAGS:
    currBroadenFunct = createBroadenFunctFromEnergiesIntensitiesAndFWHM( obj.energies, obj.baseIntensities, FWHM)
    allBroadenFuncts.append( currBroadenFunct )
    

**Modify the areas of the broadening functions by the cross-sections**

In [ ]:
xSectDatabase = yhLindau.YehLindauXSectionDatabase()
xSectCalculator = xSectObjs.CrossSectionCalculatorStandard(xSectDatabase)


In [ ]:
allLabelStrs, allXSections, allHv = list(), list(), list()
for obj in ALL_FRAGS:
    allLabelStrs.append( obj.label.xSectionLabel )
    currXSection = xSectCalculator.calculateTotalCrossSection(obj.label.xSectionLabel, PHOTON_ENERGY)
    currHv = xSectCalculator.getHvUsedToCalculateCrossSection(obj.label.xSectionLabel, PHOTON_ENERGY)
    allHv.append(currHv), allXSections.append(currXSection)

In [ ]:
#Check we're using same hv for all
hvDiffs = [abs(x-allHv[0]) for x in allHv]
assert all([x<1.0 for x in hvDiffs])

#Print out the hv and cross-section values
for hv,label,xSection in it.zip_longest(allHv,allLabelStrs, allXSections):
    print("hv = {}, {}={}".format(hv,label,xSection))

In [ ]:
#Now actually apply these cross-sections to the broadening functions. Note this would be very messy if 
#broadening functions wernt separate for different ao-fragments
def _getBFunctsModdedByXSections(bFuncts,xSections):
    outBFuncts = copy.deepcopy(bFuncts)
    for bFunct, xSection in it.zip_longest(outBFuncts,allXSections):
        currIntensities = bFunct.areas
        bFunct.areas = [x*xSection for x in currIntensities]
    return outBFuncts

finalBroadFuncts =  _getBFunctsModdedByXSections(allBroadenFuncts, allXSections)


**Generate the spectra**

In [ ]:
allLabels = [x.label for x in ALL_FRAGS]
allInpObjs = list()
for lab, bFunct in it.zip_longest(allLabels,finalBroadFuncts):
    currObj = genSpectraObjs.GenSpectraInputStandard(bFunct,lab)
    allInpObjs.append(currObj)
compositeInputObj = genSpectraObjs.GenSpectraInputCompositeStandard(allInpObjs)
spectraOutput = genSpectraRunner.getSpectrumFromApplyingBroadeningFuncts(X_VALS,compositeInputObj)
print(max(spectraOutput.totalSpectralContributions,key=lambda x:x[1]))

**Plot the spectra**

In [ ]:
dataPlotter = dPlotters.DataPlotterXPS_Standard()
allData = [spectraOutput.totalSpectralContributions]
allData.extend( spectraOutput.spectralContributions )

allDataLabels = ["Total"]
allDataLabels.extend( ["-".join(x.components) for x in spectraOutput.label] )

In [ ]:
outFig = dataPlotter.createPlot(allData, dataLabels = allDataLabels, legend=True, xLim=[0,10])